In [54]:
#Sytem
import sys,os
import datetime
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV,StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from skopt import BayesSearchCV
import xgboost as xgb
from xgboost import XGBClassifier




print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)
Target = "Perished"
PATH_LOG = "./log.csv"
PATH_df = "./df"
ID = 11
SEED = 42

----------------------------------------
Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.22.4
scikit-learn version: 0.24.2
----------------------------------------


In [16]:
df_train = pd.read_csv('../data/input/train_alpha.csv')
df_test = pd.read_csv('../data/input/test_alpha.csv')
sample = pd.read_csv('../data/input/gender_submission.csv')
data = pd.concat([df_train,df_test],sort=False)
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)

-------------------------
Shape of Train Data
[row :891,column :12]
-------------------------
Shape of Test Data
[row :418,column :12]
-------------------------


In [21]:
label = LabelEncoder()
ohe = OneHotEncoder()
data['Title'] = data['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]

title_names = (data.Title.value_counts() < 10)
data["Title"] = data["Title"].apply(lambda x: "other" if title_names.loc[x] == True else x)
faremean = data.Fare.mean()
data["Fare"].fillna(faremean,inplace=True)
data['FareBin'] = pd.qcut(data['Fare'], 10)
data['Fare_Code'] = label.fit_transform(data['FareBin'])
N_Age_Code = 3
data.replace({'Sex':{'male':0,'female':1}},inplace=True)
data['Age_Code'] = label.fit_transform(pd.qcut(data['Age'], N_Age_Code))
data['Embarked_Code'] = label.fit_transform(data['Embarked'])
data["Embarked"].fillna('S',inplace=True)
embarked_ohe = pd.get_dummies(data.Embarked)
data = pd.concat([data, embarked_ohe], axis=1)

In [22]:
data.columns

Index(['PassengerId', 'Perished', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'FareBin',
       'Fare_Code', 'Age_Code', 'Embarked_Code', 'C', 'Q', 'S'],
      dtype='object')

In [40]:
drop_columns = ["Name","Ticket","Cabin","Embarked","Title",'FareBin',"Fare_Code",'C','Q','S']
dataset = data.drop(columns=drop_columns)
train = dataset[:len(df_train)]
test = dataset[len(df_train):]
testIds = test['PassengerId'].copy()

In [25]:
X_train = train.iloc[:, 2:].values
y_train = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values

In [6]:
"""
各1層目モデルに対する操作（定義・訓練・予測）を簡略化するため、分類器の拡張クラスを定義します。
"""
class ClfBuilder(object):
    def __init__(self, clf, params=None):
        self.clf = clf(**params)

    def fit(self, X, y):
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [31]:
NUM_CLASSES = 2
"""
スタッキングでは2層目のモデルに1層目のモデルの予測値を利用します。2層目で既知のデータに対する過学習を防ぐため、1層目でOut-of-Foldによる予測値を算出してこれを2層目の学習に利用します。以下の実装では、StratifiedKFoldで5分割の交差検証を行っています。
"""
def get_base_model_preds(clf, X_train, y_train, X_test):
    print(clf.clf)

    N_SPLITS = 5
    oof_valid = np.zeros((X_train.shape[0], NUM_CLASSES))
    oof_test = np.zeros((X_test.shape[0], NUM_CLASSES))
    oof_test_skf = np.zeros((N_SPLITS, X_test.shape[0], NUM_CLASSES))

    skf = StratifiedKFold(n_splits=N_SPLITS)
    for i, (train_index, valid_index) in enumerate(skf.split(X_train, y_train)):
        print('[CV] {}/{}'.format(i+1, N_SPLITS))
        X_train_, X_valid_ = X_train[train_index], X_train[valid_index]
        y_train_, y_valid_ = y_train[train_index], y_train[valid_index]

        clf.fit(X_train_, y_train_)

        oof_valid[valid_index] = clf.predict_proba(X_valid_)
        oof_test_skf[i, :] = clf.predict_proba(X_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_valid, oof_test

In [9]:
rfc_params = {
    'n_estimators': 100, 
    'max_depth': 10, 
    'random_state': 0, 
}
gbc_params = {
    'n_estimators': 50, 
    'max_depth': 10, 
    'random_state': 0, 
}
etc_params = {
    'n_estimators': 100, 
    'max_depth': 10,
    'random_state': 0, 
}
xgbc1_params = {
    'n_estimators': 100, 
    'max_depth': 10,
    'random_state': 0, 
}
knn1_params = {'n_neighbors': 4}
knn2_params = {'n_neighbors': 8}
knn3_params = {'n_neighbors': 16}
knn4_params = {'n_neighbors': 32}

In [11]:
rfc = ClfBuilder(clf=RandomForestClassifier, params=rfc_params)
gbc = ClfBuilder(clf=GradientBoostingClassifier, params=gbc_params)
etc = ClfBuilder(clf=ExtraTreesClassifier, params=etc_params)
xgbc1 = ClfBuilder(clf=XGBClassifier, params=xgbc1_params)
knn1 = ClfBuilder(clf=KNeighborsClassifier, params=knn1_params)
knn2 = ClfBuilder(clf=KNeighborsClassifier, params=knn2_params)
knn3 = ClfBuilder(clf=KNeighborsClassifier, params=knn3_params)
knn4 = ClfBuilder(clf=KNeighborsClassifier, params=knn4_params)

In [32]:
oof_valid_rfc, oof_test_rfc = get_base_model_preds(rfc, X_train, y_train, X_test)
oof_valid_gbc, oof_test_gbc = get_base_model_preds(gbc, X_train, y_train, X_test)
oof_valid_etc, oof_test_etc = get_base_model_preds(etc, X_train, y_train, X_test)
oof_valid_xgbc1, oof_test_xgbc1 = get_base_model_preds(xgbc1, X_train, y_train, X_test)
oof_valid_knn1, oof_test_knn1 = get_base_model_preds(knn1, X_train, y_train, X_test)
oof_valid_knn2, oof_test_knn2 = get_base_model_preds(knn2, X_train, y_train, X_test)
oof_valid_knn3, oof_test_knn3 = get_base_model_preds(knn3, X_train, y_train, X_test)
oof_valid_knn4, oof_test_knn4 = get_base_model_preds(knn4, X_train, y_train, X_test)

RandomForestClassifier(max_depth=10, random_state=0)
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
GradientBoostingClassifier(max_depth=10, n_estimators=50, random_state=0)
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
ExtraTreesClassifier(max_depth=10, random_state=0)
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=0,
   

In [33]:
X_train_base = np.concatenate([oof_valid_rfc, 
                               oof_valid_gbc, 
                               oof_valid_etc, 
                               oof_valid_xgbc1, 
                               oof_valid_knn1, 
                               oof_valid_knn2, 
                               oof_valid_knn3, 
                               oof_valid_knn4, 
                              ], axis=1)
X_test_base = np.concatenate([oof_test_rfc, 
                              oof_test_gbc, 
                              oof_test_etc, 
                              oof_test_xgbc1, 
                              oof_test_knn1, 
                              oof_test_knn2, 
                              oof_test_knn3, 
                              oof_test_knn4, 
                             ], axis=1)

In [34]:
xgbc2_params = {
    'n_eetimators': 100, 
    'max_depth': 5, 
    'random_state': 42, 
}
xgbc2 = XGBClassifier(**xgbc2_params)

In [35]:
xgbc2.fit(X_train_base, y_train)

[14:38:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "n_eetimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_eetimators=100,
              n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=42, reg_alpha=0, ...)

In [50]:
prediction = xgbc2.predict(X_test_base)

In [55]:
PATH_sub =  "../data/output/"+str(ID)+"_submission_"+"ensemble"+".csv"
print(PATH_sub)
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df[Target] = prediction
submission_df.reset_index(drop=True, inplace=True)
submission_df.to_csv(PATH_sub, header=True, index=False)
print("Submission Format:{} ".format(sample.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df.head(5)

../data/output/11_submission_ensemble.csv
Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Perished
0,892,1
1,893,1
2,894,1
3,895,1
4,896,0
